# Uniview module for Supernovae over time

Data downloaded from The Open Supernova Catalog https://sne.space on Aug. 20, 2019

LSST data from [PLAsTiCC Classification Challenge](https://zenodo.org/record/2539456#.XX-1apNKhTY).  Since this file is huge (1e6 SNe per year), I will only include one month of data -- Nov. 2023.  The user has an option to show both the past and future SN together with "showBoth", and the year separation.

*Aaron Geller, 2019*

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [19]:
Template = """mesh
{  
    data SNdata ./modules/SNvTime/SNdata.raw
    #data SNdata ./modules/SNvTime/SNdataSmall.raw
    data  quad ./modules/SNvTime/quad.3ds
    data  quad2 ./modules/SNvTime/ScreenAlignedQuad.3ds

    dataHints SNdata disablePostprocessing
    
    cullRadius 10000.0
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f radiusScale 1 | public | desc "point size scale" | widget slider | range 0 10
            vec1f SNduration 0.1 | public | desc "duration of SN light curve" | widget slider | range 0 100
            vec1f SNtmin 1885 | public | desc "minimum time to show the SN" | widget slider | range 1885 2020
            vec1f SNangleMax 1.5 | public | desc "max angular size of SN circle" | widget slider | range 0 30
            vec1f SNangleMin 0.1 | public | desc "min angular size of SN circle" | widget slider | range 0 3
            bool fadeOutSN true | public  | desc "fade off SN"     
            bool showBoth false | public  | desc "show both past and future SNe"     
            vec1f bothYr 5 | public  | desc "year offset between past and future SNe" | widget slider | range 5 60

            bool useUniviewTime false | public  | desc "use Uniview time?" 
            
            vec1f eventTime 2020 | public | desc "event time " | widget slider | range 1885 2024
            vec1f transitionLength 5 | public | desc "transition length in seconds" 
            bool jump true | public  | desc "jump to time without transition"     
            
            # Properties for the date counter
            vec1f counterAzimuth 0.0  | public | desc "azimuth of the counter in degrees" | widget slider | range  -180 180             
            vec1f counterAltitude 10.0  | public |  desc "altitude of the counter in degrees" | widget slider | range  -30 30      
            vec2f counterSize 20 6  | public | desc "angular size of the counter in degrees"     
            vec1f counterDepth 0.01  | public | desc "depth of the counter in 3d space"
            vec3f counterColor 1 1 1  | public | desc "color of the counter" | widget color_rgb
            vec1f counterAlpha 1  | public | desc "opacity of the counter" | widget slider | range  0 1  

        }
    }

    ############# to hold the time information for SN
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/SNvTime/pass0.vs
                fragmentShader ./modules/SNvTime/state.fs
                textureFBO stateTexture stateTexture
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                parameter2f timeRange 1885 2024
           }
        }      
    }
    
    
    ############# SN v Time
    pass
    {
        useDataObject SNdata
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/SNvTime/SNvtime.gs
                vertexShader ./modules/SNvTime/SNvtime.vs
                fragmentShader   ./modules/SNvTime/SNvtime.fs
                textureFBO stateTexture stateTexture
                #texture cmap1 ./modules/SNvTime/cmap1.png
                #{             
                #    wrapModeS GL_CLAMP_TO_EDGE
                #    wrapModeR GL_CLAMP_TO_EDGE
                #    colorspace linear
                #}
                #texture cmap2 ./modules/SNvTime/cmap2.png
                #{             
                #    wrapModeS GL_CLAMP_TO_EDGE
                #    wrapModeR GL_CLAMP_TO_EDGE
                #    colorspace linear
                #}                
                stateManagerVar __objectName__.radiusScale  radiusScale
                stateManagerVar __objectName__.SNduration  SNduration
                stateManagerVar __objectName__.SNtmin  SNtmin
                stateManagerVar __objectName__.SNangleMax  SNangleMax
                stateManagerVar __objectName__.SNangleMin  SNangleMin
                stateManagerVar __objectName__.fadeOutSN  fadeOutSN
                stateManagerVar __objectName__.showBoth  showBoth
                stateManagerVar __objectName__.bothYr  bothYr
                stateManagerVar __objectName__.useUniviewTime  useUniviewTime
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
 
    ############# Time label
    pass 
    {
        passEnable true
        passScale 2e3 #it looks like smaller values of this allow it to be viewed closer to Earth
        useDataObject quad2
        shader
        {
            type defaultMeshShader
            {
                vertexShader      ./modules/SNvTime/imagePass.vs
                fragmentShader    ./modules/SNvTime/numbersPass.fs
                textureFBO stateTexture stateTexture
                                
                stateManagerVar __objectName__.counterAzimuth azimuth
                stateManagerVar __objectName__.counterAltitude altitude
                stateManagerVar __objectName__.counterSize imageQuadSize
                stateManagerVar __objectName__.counterDepth imageDepth
                stateManagerVar __objectName__.counterColor imageColor
                stateManagerVar __objectName__.counterAlpha imageAlpha
                stateManagerVar __objectName__.Alpha uv_alpha
                stateManagerVar __objectName__.useUniviewTime  useUniviewTime
    
                texture numbersTex ./modules/SNvTime/BigNumbers.dds
                {
                }  
     
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_POLY_FRONT_AND_BACK_MODE GL_FILL
                }   
            }
        }   
    }
    
}"""

### SN class

In [20]:
class SNClass():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("SNvtime.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNvtime.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SNvtime.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("state.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("pass0.vs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [21]:
model = SNClass(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [22]:
scene.setname("SNvTime")
scene.setparent("Extragalactic")
scene.setunit(3.09e22) #km to Mpc
#scene.setunit(1.0e6) #pc to Mpc
#scene.setunit(1) #already in units of Mpc?
#scene.setentrydist(1e5)

model.object.setcameraradius(1e-3) #near plane clipping?
model.object.setcoord(scene.name)
model.object.setname("SNvTime")
model.object.setguiname("/KavliLecture/Zooniverse/SNvTime")
model.object.settargetradius(100)
model.object.showatstartup(False)
model.cr = 1e4 #far plane clipping?

modinfo.setname("SNvTime")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Laura Trouille<sup>1</sup>, Chris Lintott<sup>2</sup> <br />(1)Adler Planetarium,<br />(2)Oxford University,")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for a SN vs time.  Data are from The Open Supernova Catalog: https://sne.space , and the PLAsTiCC Classification Challenge : https://zenodo.org/record/2539456#.XX-1apNKhTY .")
modinfo.setthumbnail("data/SNvTimeSnapshot.png")
modinfo.setversion("1.0")

generator.generate("SNvTime",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")


## To reload

In [6]:
from config import Settings

In [18]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Create colormap texture

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [26]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap, fname):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig(fname, bbox_inches='tight',pad_inches=0)

#plot_cmap('tab20')
plot_cmap('cool',"data/cmap1.png")
plot_cmap('hot',"data/cmap2.png")